In [1]:
%load_ext autoreload
%autoreload 2

import datajoint as dj
dj.config["enable_python_native_blobs"] = True
dj.config['nnfabrik.schema_name'] = "nnfabrik_v1_tuning"
schema = dj.schema("nnfabrik_v1_tuning")

from nndichromacy.tables.from_mei import TrainedEnsembleModel
from nndichromacy.tables.from_mei import MEISelector

from nnfabrik.main import *

from insilico_stimuli.tables.templates import ExperimentPerUnitTemplate
from insilico_stimuli.tables.main import ExperimentMethod, InsilicoStimuliSet

from insilico_stimuli.parameters import *


Connecting pawelp@134.2.168.16:3306


In [2]:
method_fn = 'insilico_stimuli.optimizers.bayesian_search'
method_config = dict(total_trials=30)

experiment_method = ExperimentMethod().add_method(method_fn, method_config, 'gradient descent demo', skip_duplicates=True)
method_hash = experiment_method['method_hash']

/src/nnidentify/lib/insilico-stimuli/insilico_stimuli/tables/main.py:143: UserWarning: Corresponding entry found. Skipping...
  warnings.warn("Corresponding entry found. Skipping...")


In [3]:
ExperimentMethod()

method_fn name of the set function,method_hash hash of the set config,method_config set configuration object,method_ts UTZ timestamp at time of insertion,method_comment a short comment describing the set
insilico_stimuli.optimizers.bayesian_search,84a37be1b0d0caccd25970c970ebb614,=BLOB=,2021-05-25 17:26:06,gradient descent demo
insilico_stimuli.optimizers.bruteforce,34a3bdf5633d49c4e69c3ea63319b7ee,=BLOB=,2020-12-17 15:50:59,bf_batch_1000
insilico_stimuli.optimizers.bruteforce,4371c4abc4b8db0e5112776ade52cee7,=BLOB=,2021-05-01 11:55:03,bruteforce demo
insilico_stimuli.optimizers.bruteforce,531a53f0acae47bfa0c67acb2db6bd67,=BLOB=,2020-12-16 20:36:08,bf_batch_100
insilico_stimuli.optimizers.gradient_descent,4dfd6849154d32911f0f1229dfd9e505,=BLOB=,2021-01-15 16:22:15,grad_ep_10000_lr_0.01_std_0.1
insilico_stimuli.optimizers.gradient_descent,83547a0aba34d1a03c1f77378e2c5872,=BLOB=,2020-12-29 12:18:49,grad_ep_1000_lr_0.01_std_0.1
insilico_stimuli.optimizers.gradient_descent,94ac98aa03ef6092ce3b426a3e03ee93,=BLOB=,2021-02-06 19:58:50,"ep:10k, lr: 0.01, norm: 6.9"
insilico_stimuli.optimizers.gradient_descent,bd0cb7b70119b16577d87df696535d72,=BLOB=,2021-04-22 14:29:24,gradient descent demo
insilico_stimuli.optimizers.gradient_descent,e63e52df27ccc3c48d0b6153226d72da,=BLOB=,2021-01-21 11:55:09,"ep: 10k, lr: 0.01, norm: 10"
insilico_stimuli.optimizers.mse,4eddfbfba0262d08875860ffa23112e6,=BLOB=,2021-01-27 09:33:20,"ep: 10k, lr: 0.01, norm: 10"


In [36]:
stimuli_fn = 'insilico_stimuli.stimuli.Gabor'
stimuli_config = {
    "canvas_size": {
        "args": [36, 64]
    },
    "theta": {
        "path": "insilico_stimuli.parameters.FiniteParameter",
        "args": [[0., np.pi/2, np.pi]],
    },
    "sigma": {
        "path": "insilico_stimuli.parameters.FiniteParameter",
        "args": [[5., 6., 7.]],
    },
    "Lambda": {
        "path": "insilico_stimuli.parameters.FiniteParameter",
        "args": [[9., 9.5, 10.]],
    },
    "psi": {
        "path": "insilico_stimuli.parameters.FiniteParameter",
        "args": [[0., np.pi/2, np.pi]],
    },
    "gamma": {
        "path": "insilico_stimuli.parameters.FiniteParameter",
        "args": [[1.]],
    },
    "center": {
        "path": "insilico_stimuli.parameters.FiniteParameter",
        "args": [[[0., 0.], [10., 10.], [-10., -10.]]]
    },
}

In [37]:
stimulus_set = InsilicoStimuliSet().add_set(stimuli_fn, stimuli_config, 'Gabor Bayesian Search Demo Stimulus', skip_duplicates=True)
stimulus_hash = stimulus_set['stimulus_hash']

In [35]:
(InsilicoStimuliSet() & dict(stimulus_hash=stimulus_hash)).delete()

About to delete:
`nnfabrik_v1_tuning`.`#insilico_stimuli_set`: 1 items


Proceed? [yes, No]:  yes


Committed.


In [22]:
InsilicoStimuliSet()

stimulus_fn name of the set function,stimulus_hash hash of the set config,stimulus_config set configuration object,stimulus_ts UTZ timestamp at time of insertion,stimulus_comment a short comment describing the set
insilico_stimuli.stimuli.Gabor,37debb39c75a370e2063612ee258bebe,=BLOB=,2021-05-01 11:56:39,Gabor Demo Stimulus
insilico_stimuli.stimuli.Gabor,3a4e901830b2d204477dbb5bfe3b258b,=BLOB=,2021-05-25 18:05:47,Gabor Bayesian Search Demo Stimulus
insilico_stimuli.stimuli.Gabor,6d98b52baf9189208043223589a6faf1,=BLOB=,2021-04-24 15:04:10,Gabor Stimulus
insilico_stimuli.stimuli.Gabor,b159b360714a6e6e07394acf437ddd78,=BLOB=,2021-05-01 11:32:21,Gabor Gradient Descent Demo Stimulus
insilico_stimuli.stimuli.GaborSet,292065e67b880064f146236f72bf3c17,=BLOB=,2021-01-15 16:10:49,GaborSet Gradient Descent
insilico_stimuli.stimuli.GaborSet,9c652301ee7f1573c39b2d03c07e9703,=BLOB=,2020-12-21 13:38:55,GaborSet Bruteforce Search Larger 480k
insilico_stimuli.stimuli.GaborSet,a9dd055cd11622a05392b6fc856acc73,=BLOB=,2020-12-10 18:12:08,GaborSet Orientation Tunning
insilico_stimuli.stimuli.GaborSet,b33c6da1c7ba976e4e83f85cc6c2e761,=BLOB=,2020-12-18 10:55:25,GaborSet Bruteforce Search Larger
insilico_stimuli.stimuli.GaborSet,c546cbbb50ab26fe218491c9e55ce761,=BLOB=,2020-12-29 12:17:12,GaborSet GD InitParams
insilico_stimuli.stimuli.GaborSet,c58caeddaf40883905ab17e3ca449fbd,=BLOB=,2020-12-21 13:49:16,GaborSet Bruteforce Search 480k


In [23]:
@schema
class DemoGaborStimuli(ExperimentPerUnitTemplate, dj.Computed):
    trained_model_table = TrainedEnsembleModel
    unit_table = MEISelector
    method_table = ExperimentMethod


In [24]:
DemoGaborStimuli()

method_fn name of the set function,method_hash hash of the set config,stimulus_fn name of the set function,stimulus_hash hash of the set config,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,ensemble_hash the hash of the ensemble,unit_id unique neuron identifier,data_key unique session identifier,seed experiment seed,output,score
insilico_stimuli.optimizers.bruteforce,4371c4abc4b8db0e5112776ade52cee7,insilico_stimuli.stimuli.Gabor,37debb39c75a370e2063612ee258bebe,nndichromacy.datasets.static_loaders,0d2f4b989e3cf37664080300ffe3f732,177e89418565118975d69f513c91d3ef,1,22564-2-13-0,1000,=BLOB=,1.01047
insilico_stimuli.optimizers.gradient_descent,bd0cb7b70119b16577d87df696535d72,insilico_stimuli.stimuli.Gabor,37debb39c75a370e2063612ee258bebe,nndichromacy.datasets.static_loaders,0d2f4b989e3cf37664080300ffe3f732,177e89418565118975d69f513c91d3ef,1,22564-2-13-0,1000,=BLOB=,2.04269


In [25]:
TrainedEnsembleModel()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,ensemble_hash the hash of the ensemble,ensemble_comment a short comment describing the ensemble
nndichromacy.datasets.static_loaders,0d2f4b989e3cf37664080300ffe3f732,177e89418565118975d69f513c91d3ef,closed-loop mouse ensemble 2 - imagenet
nndichromacy.datasets.static_loaders,29c0d71805fb74237c3fbed16e7f1e04,2072ec448995cbc62d990ecfaef3ccad,default 5 models
nndichromacy.datasets.static_loaders,29c0d71805fb74237c3fbed16e7f1e04,29f8ed5574b34f937eb1644f2984e926,default 5 models
nndichromacy.datasets.static_loaders,3390ff21bea702ae0dcdbcbfefb78e5e,d3eee24b439f842c9962777da4a7f0c5,closed-loop mouse ensemble
nndichromacy.datasets.static_loaders,376aabbf8c88cf0e31578719c67ed629,6b1c7d5b4f2c5f203974963a43c31f13,default 5 models
nndichromacy.datasets.static_loaders,376aabbf8c88cf0e31578719c67ed629,e0d6191d7fb620c010971bfa9cfcf9f0,default 5 models
nndichromacy.datasets.static_loaders,460f80e59cfde6ad5dca4b2c59caba3e,97e2057bbafe6d713575d0527518276b,closed-loop mouse ensemble 2 - imagenet
nndichromacy.datasets.static_loaders,52d2cbb544433e321d2e31e4cf7c199b,771ad5663b9fa4007e5eefde3c546803,window 50
nndichromacy.datasets.static_loaders,6523026e601b6b3e6c1e8d0aa5488a66,32749f84d9daf35b74fe449b756f5cf3,window 250
nndichromacy.datasets.static_loaders,72930dd1be6c229df4be82f74803262c,01c6599fe68c3e3107778f84c0aa5323,default 5 models


In [55]:
selection = dict(
    ensemble_hash='177e89418565118975d69f513c91d3ef',
    dataset_hash='0d2f4b989e3cf37664080300ffe3f732',
    stimulus_hash=stimulus_hash,
    method_hash=method_hash,
    unit_id=1,
    seed=1000
)

In [56]:
DemoGaborStimuli.populate(selection)

In [57]:
DemoGaborStimuli()

method_fn name of the set function,method_hash hash of the set config,stimulus_fn name of the set function,stimulus_hash hash of the set config,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,ensemble_hash the hash of the ensemble,unit_id unique neuron identifier,data_key unique session identifier,seed experiment seed,output,score
insilico_stimuli.optimizers.bayesian_search,84a37be1b0d0caccd25970c970ebb614,insilico_stimuli.stimuli.Gabor,31aba497fa6b512079f98f2ecbf7e27d,nndichromacy.datasets.static_loaders,0d2f4b989e3cf37664080300ffe3f732,177e89418565118975d69f513c91d3ef,1,22564-2-13-0,1000,=BLOB=,0.997244
insilico_stimuli.optimizers.bruteforce,4371c4abc4b8db0e5112776ade52cee7,insilico_stimuli.stimuli.Gabor,37debb39c75a370e2063612ee258bebe,nndichromacy.datasets.static_loaders,0d2f4b989e3cf37664080300ffe3f732,177e89418565118975d69f513c91d3ef,1,22564-2-13-0,1000,=BLOB=,1.01047
insilico_stimuli.optimizers.gradient_descent,bd0cb7b70119b16577d87df696535d72,insilico_stimuli.stimuli.Gabor,37debb39c75a370e2063612ee258bebe,nndichromacy.datasets.static_loaders,0d2f4b989e3cf37664080300ffe3f732,177e89418565118975d69f513c91d3ef,1,22564-2-13-0,1000,=BLOB=,2.04269
